In [ ]:
import logging
import random

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as plt_colors
from matplotlib.axes._axes import _log as matplotlib_axes_logger
matplotlib_axes_logger.setLevel('ERROR')
import scipy.stats as st
import holoviews as hv
hv.extension('bokeh')
from holoviews import dim
from IPython.display import Markdown, display
from IPython.core.display import HTML

import matplotlib
matplotlib.rc('xtick', labelsize=14)     
matplotlib.rc('ytick', labelsize=14)
matplotlib.rc('axes', labelsize=14, titlesize=14)

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import hvplot.pandas

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import summary_table

from counts_analysis.c_utils import COUNTS_CSV, CLASSES, set_settings, set_counts_v2, rename_columns

#== Load Datasets ==#
df = pd.read_csv(COUNTS_CSV['counts'])
df = rename_columns(df)
# Dataset without problematic classes (Gyrodinium, Pseudo-nitzchia chain)
df_ = df[df['class'].isin(CLASSES)].reset_index(drop=True)
data = df.copy()

def printmd(string):
    display(Markdown(string))

#=== Set count forms & settings ===#
# COUNT
volumetric_counts = set_counts_v2('cells/mL', micro_default=True)

raw_counts = set_counts_v2('count', micro_default=False)
raw_counts_pred = set_counts_v2('count', micro_default=False, automated=True)

vol_time_counts = set_counts_v2('count', micro_default=True)

class_percentage_counts = set_counts_v2('class percentage', micro_default=False) 

rel_counts = set_counts_v2('relative abundance', micro_default=False)
rel_counts = ['Lab-micro count relative abundance'] + list(rel_counts[1:])

class_percentage_counts_pred = set_counts_v2('class percentage', micro_default=False, automated=True)
class_percentage_counts_pred = ['Lab-micro class percentage'] + list(class_percentage_counts_pred[1:])

# HOLOVIEWS Regression Plot
Main issues: cannot plot all points in logx, logy due to error in the range

In [ ]:
def get_linear_fit(counts, data, color, fontscale=1.5, line_width=5):
    x = data[counts[0]].values
    y = data[counts[1]].values
    X = sm.add_constant(x)
    res = sm.OLS(y, X).fit()
    printmd(f'### {counts}')
    print(res.summary())
    
    st, reg_data, ss2 = summary_table(res, alpha=0.05)
    fittedvalues = reg_data[:, 2]
    predict_mean_se  = reg_data[:, 3]
    predict_mean_ci_low, predict_mean_ci_upp = reg_data[:, 4:6].T
    predict_ci_low, predict_ci_upp = reg_data[:, 6:8].T
    
    X = X[:,1]
    reg = hv.Curve(list(zip(X, fittedvalues)), label='Linear Fit').opts(tools=['hover'], color=color, fontscale=fontscale, line_width=line_width, alpha=0.8)
    predict_ci = hv.Area((X, predict_ci_low, predict_ci_upp), vdims=['y', 'y2'], label='95% Confidence Band').opts(alpha=0.2, color=color)
    predict_mean_ci = hv.Area((X, predict_mean_ci_low, predict_mean_ci_upp), vdims=['y', 'y2'], label='95% Prediction Band').opts(alpha=0.4, color=color)
    
    return reg, predict_ci, predict_mean_ci

def plot_class_summary_2method(counts, data, relative=False, color=None):
    """ Plot individual summaries of each class

    Usage

    >>> plot_class_summary(rc_counts, cls_df)

    Args:
        counts:
        data:
        relative:

    Returns:

    """
    fontscale = 1.5
    title_pre = '[{}]'.format(cls) if not relative else '[Relative Abundance]'
    xy = 'Count' if not relative else 'Relative Abundance'
    max_val = max(data[list(counts)].max()) + 10

    # correlation plot
    dot_size, alpha = 8, 0.6
    line_width = 5

    # Plot points onto scatter plot
    sc1 = hv.Scatter(data, counts[0], [counts[1], 'datetime', 'class'], label='Raw Data').opts(size=dot_size, tools=['hover'], color=color[-1], fontscale=fontscale)
    # Get linear regression fit
    reg, predict_ci, predict_mean_ci = get_linear_fit(counts, data, color=color[-1], fontscale=1.5, line_width=5)
    # Plot all curves on one layout
    corr = (sc1*reg * predict_ci * predict_mean_ci).opts(xlabel=counts[0] , ylabel=counts[1],
                                            title=f'{title_pre} Correlation', xlim=(0, max_val), ylim=(0, max_val), tools=['hover'], width=650, height=500, legend_position='right')

    cls_plot = hv.Layout(corr).cols(3)

    return cls_plot

cls = 'Prorocentrum micans'
x_data = data[data['class'] == cls].reset_index(drop=True)
plot_class_summary_2method([raw_counts[0], raw_counts[1]], x_data, relative=False, color=['blue', 'orange', 'black'])

## Attempt at Plotting Regression Plot with Log scale w/out confidence bands

In [ ]:
%%opts Scatter [tools=['hover'], width=300, height=300, legend_position='right', xlim=(-1, None), ylim=(-1, None), logx=True, logy=True]
%%opts Slope [xlim=(-1, None), ylim=(-1, None), logx=True, logy=True]

def plot_correlation_hv(counts, data, cls, relative=False):
    title_pre = '[Absolute Count]' if not relative else '[Relative Abundance]'
    xy = 'Count' if not relative else 'Relative Abundance'
    max_val = max(data[list(counts)].max()) + 10
    
    # correlation plot
    dot_size, alpha = 12, 0.6
    fs = 18

    sc1 = hv.Scatter(data, counts[0], [counts[1], 'datetime', 'class'],
                     label='lab - micro').opts(size=dot_size, alpha=alpha,
                                               tools=['hover'])
    reg = hv.Slope.from_scatter(sc1).opts(alpha=alpha, tools=['hover'], )

    sc2 = hv.Scatter(data, counts[0], [counts[2], 'datetime', 'class'],
                     label='pier - micro').opts(size=dot_size, alpha=alpha,
                                                tools=['hover'], )
    reg2 = hv.Slope.from_scatter(sc2).opts(alpha=alpha, tools=['hover'], )

    sc3 = hv.Scatter(data, counts[1], [counts[2], 'datetime', 'class'],
                     label='pier - lab').opts(size=dot_size, alpha=alpha,
                                              tools=['hover'], )
    reg3 = hv.Slope.from_scatter(sc3).opts(alpha=alpha, tools=['hover'], )
    
    corr = (sc1 * sc2 * sc3 * reg * reg2 * reg3).opts(xlabel=xy, ylabel=xy,
                                                      title=f'{cls}',
                                                      xlim=(0, max_val),
                                                      ylim=(0, max_val), tools=['hover'],
                                                      width=330, height=330,
#                                                       legend_position='right',
                                                      show_legend=False,
                                                      fontsize={'title': fs, 'labels': fs, 'xticks': fs, 'yticks': fs}).redim.range(y=(0.01, 1))
    return corr.redim.range(y=(0.01, None), x=(0.01, None))

y = data.copy()
cls = 'Akashiwo'
COUNT = raw_counts
def filter_classes(df, classes):
    return df[df['class'].isin(classes)].reset_index(drop=True)
cls_df = filter_classes(y, [cls])
corr = plot_correlation_hv(COUNT, cls_df, cls, True)

clsses = ['Ceratium falcatiforme or fusus', 'Ceratium furca', 'Cochlodinium', 'Lingulodinium polyedra', 'Prorocentrum micans']
# clsses = ['Ceratium furca', 'Cochlodinium', 'Lingulodinium polyedra', 'Prorocentrum micans']
for cls in clsses:
    print(cls)
    cls_df = filter_classes(y, [cls])
    corr += plot_correlation_hv(COUNT, cls_df, cls, True)
hv.Layout(corr.redim.range(y=(0.01, None), x=(0.01, None))).cols(3)

In [ ]:
"""
RUN this to get regression` plots (SPC-Pier vs. Lab-micro, SPC-Lab vs. Lab-micro, etc.) for an individual species
*** Otherwise, SKIP OR HIDE THIS ***

Example
-------
>>> cls = 'Prorocentrum micans'
>>> _plot_class_summary(cls, DATASET, DATASET, x_relative=False, y_relative=True, classifier=False)
"""
def plot_class_summary(counts, data, relative=False):
    """ Plot individual summaries of each class

    Usage

    >>> plot_class_summary(rc_counts, cls_df)

    Args:
        counts:
        data:
        relative:

    Returns:

    """
    fontscale = 1.5
    title_pre = 'Compared Counts' if not relative else '[Relative Abundance]'
    xy = 'Count' if not relative else 'Relative Abundance'
    max_val = max(data[list(counts)].max()) + 10

    # boxwhisker plot
    rot = 0 if not relative else 5
    bx = data.groupby('datetime')[counts].sum().hvplot.hist(y=list(counts),
                                                           group_label='Sampling Technique',
                                                           value_label=xy,
                                                           label='{} Distribution'.format(title_pre),
                                                           rot=rot)\
        .opts(tools=['hover'], width=400, height=500, show_legend=False, fontscale=fontscale)

    # time series
    ts = data.groupby('datetime')[counts].sum().hvplot.line(rot=30, value_label='Total Count', group_label='Sampling Techniques', label=f'{title_pre} Time Series').\
        opts(height=500, width=800, legend_position='top_right', fontscale=fontscale)

    # correlation plot
    dot_size, alpha = 8, 0.6
    line_width = 5

    sc1 = hv.Scatter(data, counts[0], [counts[1], 'datetime', 'class'], label='lab - micro').opts(size=dot_size, alpha=alpha, tools=['hover'], fontscale=fontscale, color='blue')
#     reg = hv.Slope.from_scatter(sc1).opts(alpha=alpha, tools=['hover'], fontscale=fontscale, color='blue', line_width=line_width)
    reg, predict_ci, predict_mean_ci = get_linear_fit([counts[0], counts[1]], data, color='blue', fontscale=1.5, line_width=5)
    corr1 = sc1 * reg * predict_ci * predict_mean_ci

    sc2 = hv.Scatter(data, counts[0], [counts[2], 'datetime', 'class'], label='pier - micro').opts(size=dot_size, alpha=alpha, tools=['hover'], fontscale=fontscale, color='gold')
#     reg2 = hv.Slope.from_scatter(sc2).opts(alpha=alpha, tools=['hover'], fontscale=fontscale, color='gold', line_width=line_width)
    reg, predict_ci, predict_mean_ci = get_linear_fit([counts[0], counts[2]], data, color='gold', fontscale=1.5, line_width=5)
    corr2 = sc2 * reg * predict_ci * predict_mean_ci 

    sc3 = hv.Scatter(data, counts[1], [counts[2], 'datetime', 'class'], label='pier - lab').opts(size=dot_size, alpha=alpha, tools=['hover'], fontscale=fontscale, color='red')
#     reg3 = hv.Slope.from_scatter(sc3).opts(alpha=alpha, tools=['hover'], fontscale=fontscale, color='red', line_width=line_width)
    reg, predict_ci, predict_mean_ci = get_linear_fit([counts[1], counts[2]], data, color='red', fontscale=1.5, line_width=5)
    corr3 = sc3 * reg * predict_ci * predict_mean_ci 

#     corr = sc1*sc2*sc3*reg*reg2*reg3
    corr = (corr1 * corr2 * corr3).opts(xlabel=xy , ylabel=xy,
                                            title=f'{title_pre} Correlation', xlim=(0, max_val), ylim=(0, max_val), tools=['hover'], width=650, height=500, legend_position='right')

    cls_plot = hv.Layout(bx + ts + corr).cols(3)

    return cls_plot

def _plot_class_summary(cls, x, y, x_relative=False, y_relative=True, classifier=False):
    def plot_class_summary_2method(counts, data, relative=False, color=None):
        """ Plot individual summaries of each class

        Usage

        >>> plot_class_summary(rc_counts, cls_df)

        Args:
            counts:
            data:
            relative:

        Returns:

        """
        fontscale = 1.5
        title_pre = '[{}]'.format(cls) if not relative else '[Relative Abundance]'
        xy = 'Count' if not relative else 'Relative Abundance'
        max_val = max(data[list(counts)].max()) + 10

        # boxwhisker plot
        rot = 0 if not relative else 5
        bx = data.groupby('datetime')[counts].sum().hvplot.hist(y=list(counts),
                                                               label='{} Distribution'.format(title_pre),
                                                                xlabel='Count Value', ylabel='Frequency of Count',
                                                               rot=rot,
                                                               color=color[:2])\
            .opts(tools=['hover'], width=400, height=500, fontscale=fontscale, show_legend=False)

        # time series
        ts = data.groupby('datetime')[counts].sum().hvplot.line(rot=30, value_label='Count', group_label='Sampling Techniques', label=f'{title_pre} Time Series', color=color[:2]).\
            opts(height=500, width=800, legend_position='top_right', fontscale=fontscale)

        # correlation plot
        dot_size, alpha = 8, 0.6
        line_width = 5

        sc1 = hv.Scatter(data, counts[0], [counts[1], 'datetime', 'class'], label='Raw Data').opts(size=dot_size, alpha=alpha, tools=['hover'], color=color[-1], fontscale=fontscale)
#         reg = hv.Slope.from_scatter(sc1).opts(alpha=alpha, tools=['hover'], color=color[-1], fontscale=fontscale, line_width=line_width)
        reg, predict_ci, predict_mean_ci = get_linear_fit(counts, data, color=color[-1], fontscale=1.5, line_width=5)
#         corr = (sc1*reg * predict_ci * predict_mean_ci)

        corr = (sc1*reg * predict_ci * predict_mean_ci).opts(xlabel=counts[0] , ylabel=counts[1],
                                                title=f'{title_pre} Correlation', xlim=(0, max_val), ylim=(0, max_val), tools=['hover'], width=650, height=500, legend_position='right')

        cls_plot = hv.Layout(bx + ts + corr).cols(3)

        return cls_plot
    
    def filter_class(cls, x_data, y_data):
        x_cls_df = x_data[x_data['class'] == cls].reset_index(drop=True)
        y_cls_df = y_data[y_data['class'] == cls].reset_index(drop=True)
        return x_cls_df, y_cls_df

    def plot_summary(x, y):
        x_count, x_data, x_relative = x
        y_count, y_data, y_relative = y
        datetime_col = ['datetime']    
        
        plot1 = plot_class_summary_2method([x_count[0], x_count[1]], x_data, relative=x_relative, color=['blue', 'orange', 'blue'])
        plot2 = plot_class_summary_2method([x_count[0], x_count[2]], x_data, relative=x_relative, color=['blue', 'green', 'gold'])
        plot3 = plot_class_summary_2method([x_count[1], x_count[2]], x_data, relative=x_relative, color=['orange', 'green', 'red'])
        
        plot4 = plot_class_summary(x_count, x_data, relative=x_relative)

        return hv.Layout(plot4 + plot1 + plot2 + plot3 ).cols(3).opts(shared_axes=False)

    x_df, y_df = filter_class(cls, x, y)
    printmd(f'# {cls} | Classifier Counts ({classifier})')
    if classifier:
        x_counts, y_counts = raw_counts_pred, class_percentage_counts_pred
    else:
        x_counts, y_counts = raw_counts, class_percentage_counts
    return plot_summary((x_counts, x_df, x_relative), (y_counts, y_df, y_relative))

# %%opts Scatter [tools=['hover'], width=600, height=600, legend_position='right', logx=True, logy=True, xlim=(-1, None), ylim=(-1, None)]
# %%opts Slope [logx=True, logy=True, xlim=(-1, None), ylim=(-1, None)]
DATASET = data
cls = 'Prorocentrum micans'
_plot_class_summary(cls, DATASET, DATASET, x_relative=False, y_relative=True, classifier=False)

# Matplotlib Regression Plot
Main issues: cannot plot linear regression line that works with logx, logy scale

In [ ]:
matplotlib.rc('xtick', labelsize=10)     
matplotlib.rc('ytick', labelsize=10)
matplotlib.rc('axes', labelsize=10)

current_palette_7 = sns.color_palette("Set2", 3)
sns.set_palette(current_palette_7[::-1])

def rsquared(x, y):
    """ Return R^2 where x and y are array-like."""
    import statsmodels.api as sm
    X = np.hstack((np.array([1] * len(x)).reshape(-1, 1), np.array(x).reshape(-1, 1)))
    mod = sm.OLS(np.array(y).reshape(-1, 1), X)
    res = mod.fit()
    return res

def best_fit(X, Y, log_scale=False, verbose=False):
    if log_scale:
        # slope, intercept, \
        # r_value, p_value, std_err = linregress(np.log10(np.array(X) + 1), np.log10(np.array(Y)+1))
        # Xfit = np.logspace(-1, 4, base=10)
        # Yfit = Xfit * slope + intercept

        x1 = [x for (x, y) in sorted(zip(X, Y))]
        y1 = [y for (x, y) in sorted(zip(X, Y))]
        x = np.array([np.log(x) if x>=1 else 1 for x in x1])
        y = np.array([np.log(x) if x>=1 else 1 for x in y1])
        k,m = np.polyfit(x, y, deg=1)
        Xfit = x1
        Yfit = np.exp(m) * x1**(k)
#         Yfit = fit[0] * x + fit[1]


    else:
        xbar = sum(X) / len(X)
        ybar = sum(Y) / len(Y)
        n = len(X)  # or len(Y)

        numer = sum([xi * yi for xi, yi in zip(X, Y)]) - n * xbar * ybar
        denum = sum([xi ** 2 for xi in X]) - n * xbar ** 2

        b = numer / denum if denum !=0 else 0
        a = ybar - b * xbar

        Yfit = [a + b * xi for xi in X]
        Xfit = X

    # Compute R2 value and other statistics from statsmodel
    res = rsquared(X, Y)

    if verbose:
        print(res.summary())
    return Xfit, Yfit

def plot_correlation(data, counts, logged=False):
    NUM_COLS=5
    fig, ax = plt.subplots(2, NUM_COLS, figsize=(20, 8))
    sns.scatterplot(x=data[counts[0]], y=data[counts[1]], ax=ax[0,0], label='lab (Y) - micro (X)')
    sns.scatterplot(x=data[counts[0]], y=data[counts[2]], ax=ax[0,0], label='pier (Y) - micro (X)')
    sns.scatterplot(x=data[counts[1]], y=data[counts[2]], ax=ax[0,0], label='pier (Y) - lab (X)')

    ax[0,0].set_xlabel('Count (X)')
    ax[0,0].set_ylabel('Count (Y)')

    plt.tight_layout()
    classes = sorted(data['class'].unique())
    for i_ax,cls in enumerate(classes):
        cls_df = data[data['class']==cls]
        ax_idx = ax[int((i_ax+1) / NUM_COLS), (i_ax+1) % NUM_COLS]
        sns.scatterplot(x=cls_df[counts[0]], y=cls_df[counts[1]], ax=ax_idx, label='lab (Y) - micro (X)')
        sns.scatterplot(x=cls_df[counts[0]], y=cls_df[counts[2]], ax=ax_idx, label='pier (Y) - micro (X)')
        sns.scatterplot(x=cls_df[counts[1]], y=cls_df[counts[2]], ax=ax_idx, label='pier (Y) - lab (X)')
        Xfit, Yfit = best_fit(cls_df[counts[0]], cls_df[counts[1]], logged, verbose=False)
        ax_idx.plot(Xfit, Yfit)

        Xfit, Yfit = best_fit(cls_df[counts[0]], cls_df[counts[2]], logged, verbose=False)
        ax_idx.plot(Xfit, Yfit)

        Xfit, Yfit = best_fit(cls_df[counts[1]], cls_df[counts[2]], logged, verbose=False)
        ax_idx.plot(Xfit, Yfit)

        ax_idx.set_xlabel('Count (X)')
        ax_idx.set_ylabel('Count (Y)')
        
        ymin, ymax = ax_idx.get_ylim()
        xmin, xmax = ax_idx.get_xlim()
        
        max_val = xmax if xmax >= ymax else ymax
        ax_idx.set_ylim(0, max_val)
        ax_idx.set_xlim(0, max_val)
        ax_idx.set_yscale('symlog')
        ax_idx.set_xscale('symlog')

        ax_idx.set_title(cls)
#         set_plotting_opts(ax_idx, logged=LOGGED)
        plt.tight_layout()
    plt.show()

In [ ]:
printmd('### Absolute Counts')
plot_correlation(data, raw_counts, logged=True)

## Additional Example of Matplotlib Linear Regression plot
Example is able to plot the confidence and prediction bands somewhat correctly, but not able to achieve same results on log scale.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import summary_table

x = x_data[raw_counts[0]].values
y = x_data[raw_counts[1]].values
X = sm.add_constant(x)
res = sm.OLS(y, X).fit()
print(res.summary())

st, reg_data, ss2 = summary_table(res, alpha=0.05)
fittedvalues = reg_data[:, 2]
predict_mean_se  = reg_data[:, 3]
predict_mean_ci_low, predict_mean_ci_upp = reg_data[:, 4:6].T
predict_ci_low, predict_ci_upp = reg_data[:, 6:8].T

X = X[:,1]
fig, ax = plt.subplots(figsize=(8,6))
ax.plot(x, y, 'o', label="Raw Data")
ax.plot(X, fittedvalues, 'r-', label='Linear Fit')
ax.plot(X, predict_ci_low, 'b--')
ax.plot(X, predict_ci_upp, 'b--', label='95% Confidence Band')
ax.plot(X, predict_mean_ci_low, 'g--')
ax.plot(X, predict_mean_ci_upp, 'g--', label='95% Prediction Band')
# ax.fill_between(X, predict_mean_ci_low, predict_mean_ci_upp, color='red', alpha='0.2')
# ax.fill_between(X, predict_ci_low, predict_ci_upp, color='blue', alpha='0.2')
ax.legend(loc='best');
plt.show()